In [457]:
from random import randint, choice, random

In [458]:
def tabuleiro(tamanho, quantidade):
  return [[randint(0,tamanho-1) for j in range(tamanho)] for i in range(quantidade)]

# tabuleiro(8,10)

In [459]:
def moveRainha(l,i):
  return [l[:i] + [k] + l[i+1:] for k in range(0,len(l)) if k != l[i]]

def todosVizinhos(T):
  vizinhos = []
  for i in range(len(T)):
    vizinhos.extend(moveRainha(T,i))
  return vizinhos

# todosVizinhos(tabuleiro(8,1)[0])

In [460]:
def umVizinho(T):
  return choice(todosVizinhos(T))

# tab = tabuleiro(8,1)[0]
# print(tab)
# print(umVizinho(tab))

In [461]:
def numeroAtaques(T):
  ataques = 0
  for i in range(len(T)):
    for j in range(i+1, len(T)):
      if(T[i] == T[j]): ataques += 1#; print(f'Rainha {i} ataca horizontalmente rainha {j}')
      if(abs(T[i] - T[j]) == abs(i - j)): ataques += 1#; print(f'Rainha {i} ataca diagonalmente rainha {j}')
  return ataques

# tab = tabuleiro(8,1)[0]
# print(tab)
# print(numeroAtaques(tab))

In [462]:
def inteiro_para_gene(inteiro):
    binario = bin(inteiro)[2:]
    return '0'*(tamanho_gene - len(binario)) + binario

In [463]:
def gera_populacao(n):
  tabuleiros = tabuleiro(tamanho_tabuleiro, n)
  cromossomos = ["".join([inteiro_para_gene(i) for i in tabuleiros[j]]) for j in range(n)]
  return cromossomos

In [464]:
# gera_populacao(5)

In [465]:
def cromossomo_para_tabuleiro(cromossomo):
  tabuleiro = []
  for i in range(0, len(cromossomo), tamanho_gene):
    tabuleiro.append(int(cromossomo[i:i+tamanho_gene], 2))
  return tabuleiro

In [466]:
# cromossomo_para_tabuleiro('001111111010011010000000')

In [467]:
def adaptacao(cromossomo):
  tabuleiro = cromossomo_para_tabuleiro(cromossomo)
  maximo_ataques = tamanho_tabuleiro * (tamanho_tabuleiro - 1) / 2
  # return 1/(1 + numeroAtaques(tabuleiro))
  return (maximo_ataques - numeroAtaques(tabuleiro)) / maximo_ataques

In [468]:
# print(adaptacao('001111111010011010000000'))
# print(adaptacao('000000000000000000000000'))
# print(adaptacao('111111111111111111111111'))

In [469]:
def roleta(populacao):
  soma = sum([adaptacao(cromossomo) for cromossomo in populacao])
  probabilidade = []
  for i in range(len(populacao)):
    probabilidade.append(adaptacao(populacao[i]) / soma)
    if i > 0:
      probabilidade[i] += probabilidade[i - 1]

  return probabilidade

In [470]:
# roleta(['110000011000010111011011',
#  '100100111000010101011000',
#  '111001000000010111000011',
#  '110101010101010100010111',
#  '000000000000000000000000'])

In [471]:
def selecao(populacao):
  probabilidade = roleta(populacao)
  selecionados = []
  for i in range(len(populacao)):
    r = random()
    for j in range(len(probabilidade)):
      if r <= probabilidade[j]:
        selecionados.append(populacao[j])
        break
  return selecionados

In [472]:
# selecao(['110000011000010111011011',
#  '100100111000010101011000',
#  '111001000000010111000011',
#  '110101010101010100010111',
#  '000000000000000000000000'])

In [473]:
def crossover(pai, mae):
  corte = randint(1, tamanho_cromossomo - 1)
  filho1 = pai[:corte] + mae[corte:]
  filho2 = mae[:corte] + pai[corte:]
  return filho1, filho2

In [474]:
# crossover('000000000000000000000000', '111111111111111111111111')

In [475]:
def mutacao(cromossomo):
  posicao = randint(0, tamanho_cromossomo - 1)
  cromossomo = cromossomo[:posicao] + str(1 - int(cromossomo[posicao])) + cromossomo[posicao + 1:]
  return cromossomo

In [476]:
# mutacao('000000000000000000000000')

In [477]:
# mutacao('111111111111111111111111')

In [478]:
def seleciona_elite(populacao):
  populacao = sorted(populacao, key = adaptacao, reverse = True)
  return populacao[0]

In [479]:
# seleciona_elite(['110000011000010111011011',
#  '100100111000010101011000',
#  '111001000000010111000011',
#  '110101010101010100010111',
#  '000000000000000000000000'])

In [480]:
def algoritmoGenetico(tamanho_populacao, n_geracoes, prob_crossover, prob_mutacao, elitismo = False):
  populacao = gera_populacao(tamanho_populacao) # Gera a população inicial
  for i in range(n_geracoes):
    populacao_intermediaria = selecao(populacao) # Seleciona os indivíduos da população
    elite = seleciona_elite(populacao)
    #print(adaptacao(elite))
    #print(populacao)

    populacao = []
    for j in range(0, len(populacao_intermediaria) - 1, 2): # Realiza o crossover
      if random() < prob_crossover: 
        filho1, filho2 = crossover(populacao_intermediaria[j], populacao_intermediaria[j + 1])
        populacao.append(filho1)
        populacao.append(filho2)
      else:
        populacao.append(populacao_intermediaria[j])
        populacao.append(populacao_intermediaria[j + 1])
    
    if len(populacao_intermediaria) % 2 == 1: # Caso o número de indivíduos seja ímpar
      populacao.append(populacao_intermediaria[-1])

    for j in range(len(populacao)): # Realiza a mutação
      if random() < prob_mutacao:
        populacao[j] = mutacao(populacao[j])

    if elitismo: 
      populacao_intermediaria = populacao_intermediaria[:tamanho_populacao - 1] + [elite]

  return populacao

In [500]:
tamanho_tabuleiro = 8
tamanho_gene = len(bin(tamanho_tabuleiro - 1)[2:])
tamanho_cromossomo = tamanho_tabuleiro * tamanho_gene

In [506]:
populacao = algoritmoGenetico(60, 1500, 0.8, 0.04)
elite = seleciona_elite(populacao)
valor_elite = adaptacao(elite)
print(valor_elite)
cromossomo_para_tabuleiro(elite)

1.0


[5, 3, 1, 7, 4, 6, 0, 2]